- _purpose_: Test `/src/project_utils/satellite.fetch_esri_from_row()` and see how it breaks

- _status_: Code working. Investigation complete.

- _next_: None.

- _conclusion_:  Established that `satellite.fetch_esri_from_row()` breaks if bounding box length is smaller than 120 m. `https://services.arcgisonline/ArcGIS/rest/services/World_Imagery/MapServer/export` refuses the request and returns error 500.

In [91]:
import os
import sys

import pandas as pd
import matplotlib.pyplot as plt
import math
import numpy as np

In [61]:
sys.path.insert(0, os.path.join(os.getcwd(), "src"))

from project_utils import config as proj_config
from project_utils import satellite as proj_satellite

%load_ext autoreload
%autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
PATH_TO_CONFIG = "main/config.yml"
config = proj_config.Config(PATH_TO_CONFIG)

PATH_TO_METADATA = config.get("paths", "dataset_csv")
METADATA_FILENAME = "cms_brazil_lidar_tile_metadata.csv"

df = pd.read_csv(os.path.join(
    PATH_TO_METADATA,
    METADATA_FILENAME
    )
)

df_sorted = df.sort_values(by="tile_area_m2")

In [44]:
row = df_sorted.iloc[0]
filename = row["filename"]
area = row["tile_area_m2"]

print(filename)
print(f"{area:.4f} km**2")

PRG_A01_2013_P17a_laz_3.laz
0.0014 km**2


In [40]:
# Plot the sorted 'tile_area_m2' values
def plot_sorted_tile_areas():
    plt.figure(figsize=(12, 6))

    # Plot of 'tile_area_m2'
    plt.subplot(1, 2, 1)
    plt.plot(df_sorted['tile_area_m2'].reset_index(drop=True), marker='o')
    plt.title('Tile Area Plot')
    plt.xlabel('Index')
    plt.ylabel('Tile Area (m²)')
    plt.grid(True)

    # Cumulative plot of 'tile_area_m2'
    plt.subplot(1, 2, 2)
    plt.plot(df_sorted['tile_area_m2'].cumsum().reset_index(drop=True), marker='o')
    plt.title('Cumulative Tile Area Plot')
    plt.xlabel('Index')
    plt.ylabel('Cumulative Tile Area (m²)')
    plt.grid(True)

    # Display the plots
    plt.tight_layout()
    plt.show()

# plot_sorted_tile_areas()

In [56]:
coords = proj_satellite.get_coords(df, filename)
centre_lat, centre_lon = proj_satellite.get_centre_coord(coords)

In [57]:
img = proj_satellite.fetch_esri_from_row(df, filename)

[satellite] Downloading ESRI image for PRG_A01_2013_P17a_laz_3.laz...


HTTPError: 500 Server Error:  for url: https://services.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/export?bbox=-47.69956998%2C-3.334699031%2C-47.69956935%2C-3.334698849&bboxSR=4326&imageSR=4326&size=512%2C512&format=jpg&f=image

In [89]:
def test_esri_with_area(area, verbose=False, show=False):# Center of London approx
    centre_lat, centre_lon = 51.5, 0

    # Square of 100 km²: side = sqrt(100) = 10 km
    side_km = math.sqrt(area)
    if verbose:
        print(f"Area: {area:.3f} km**2")
        print(f"Side length: {side_km:.3f} km")
    delta_deg = side_km / 111  # ≈ 0.09 degrees

    min_lat = centre_lat - delta_deg / 2
    max_lat = centre_lat + delta_deg / 2
    min_lon = centre_lon - delta_deg / 2
    max_lon = centre_lon + delta_deg / 2

    london_bbox = (min_lat, max_lat, min_lon, max_lon)
    try:
        img = proj_satellite.fetch_esri_from_coords(london_bbox)
        if show:
            img.show()  # or however you want to display it
    except Exception as e:
        print(f"Error with area {area:.3f}; {e}")

test_esri_with_area(100000, verbose=True, show=True)

Area: 100000.000 km**2
Side length: 316.228 km


In [98]:
for area in np.arange(0.01, 0.1, 0.01):
    try:
        test_esri_with_area(area, verbose=True, show=True)
    except Exception as e:
        print(f"Error with area {area:.3f}; {e}")

Area: 0.010 km**2
Side length: 0.100 km
Error with area 0.010; 500 Server Error:  for url: https://services.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/export?bbox=-0.00045045045045045046%2C51.49954954954955%2C0.00045045045045045046%2C51.50045045045045&bboxSR=4326&imageSR=4326&size=512%2C512&format=jpg&f=image
Area: 0.020 km**2
Side length: 0.141 km
Area: 0.030 km**2
Side length: 0.173 km
Area: 0.040 km**2
Side length: 0.200 km
Area: 0.050 km**2
Side length: 0.224 km
Area: 0.060 km**2
Side length: 0.245 km
Area: 0.070 km**2
Side length: 0.265 km
Area: 0.080 km**2
Side length: 0.283 km
Area: 0.090 km**2
Side length: 0.300 km


In [106]:
def test_esri_with_width(side_km, verbose=False, show=False):# Center of London approx
    centre_lat, centre_lon = 51.5, 0

    # Square of 100 km²: side = sqrt(100) = 10 km
    area = side_km ** 2
    if verbose:
        print(f"Area: {area:.3f} km**2")
        print(f"Side length: {side_km:.3f} km")
    delta_deg = side_km / 111  # ≈ 0.09 degrees

    min_lat = centre_lat - delta_deg / 2
    max_lat = centre_lat + delta_deg / 2
    min_lon = centre_lon - delta_deg / 2
    max_lon = centre_lon + delta_deg / 2

    london_bbox = (min_lat, max_lat, min_lon, max_lon)
    try:
        img = proj_satellite.fetch_esri_from_coords(london_bbox)
        if show:
            img.show()  # or however you want to display it
    except Exception as e:
        print(f"Error with area {area:.3f}; {e}")

test_esri_with_width(0.014, verbose=True, show=True)

Area: 0.000 km**2
Side length: 0.014 km
Error with area 0.000; 500 Server Error:  for url: https://services.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/export?bbox=-6.306306306306306e-05%2C51.49993693693694%2C6.306306306306306e-05%2C51.50006306306306&bboxSR=4326&imageSR=4326&size=512%2C512&format=jpg&f=image


In [104]:
for width in np.arange(0.01, 0.21, 0.01):
    try:
        test_esri_with_width(width, verbose=True, show=True)
    except Exception as e:
        print(f"Error with area {width:.3f}; {e}")

Area: 0.000 km**2
Side length: 0.010 km
Error with area 0.000; 500 Server Error:  for url: https://services.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/export?bbox=-4.5045045045045046e-05%2C51.49995495495496%2C4.5045045045045046e-05%2C51.50004504504504&bboxSR=4326&imageSR=4326&size=512%2C512&format=jpg&f=image
Area: 0.000 km**2
Side length: 0.020 km
Error with area 0.000; 500 Server Error:  for url: https://services.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/export?bbox=-9.009009009009009e-05%2C51.49990990990991%2C9.009009009009009e-05%2C51.50009009009009&bboxSR=4326&imageSR=4326&size=512%2C512&format=jpg&f=image
Area: 0.001 km**2
Side length: 0.030 km
Error with area 0.001; 500 Server Error:  for url: https://services.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/export?bbox=-0.00013513513513513514%2C51.49986486486487%2C0.00013513513513513514%2C51.50013513513513&bboxSR=4326&imageSR=4326&size=512%2C512&format=jpg&f=image
Area: 0.002 